In [1]:
from annoy import AnnoyIndex
from tqdm import tqdm
import numpy as np
import pandas as pd

GET TABLES

In [2]:
julien_products_df = pd.read_csv("/home/jupyter/20231123_stage_vincent_products.csv", sep=",", engine="python")
julien_products_df.reset_index(inplace=True)

In [3]:
from cdiscount import config
from cdiscount import snowflake

secrets = config.load_secrets('secrets.yml', key='snowflake')
with snowflake.get_snowflake_connection(**secrets) as con:
    julien_product_seller_df = pd.read_csv("/home/jupyter/20231123_stage_vincent_product_seller_edges.csv", sep=",", engine="python")
    lr_df = snowflake.query_snowflake_to_df(
    """
   WITH CATEGORY_LEVEL4_PERIMETER AS (
        SELECT DISTINCT PRODUCT_CATEGORY_LEVEL4_ID AS PRODUCT_CATEGORY_LEVEL4_ID
        FROM QUALISCORE_LAB.LAB_STAGE_VINCENT_CANAPES_WITH_PROPERTIES
    ),
    CATEGORY_LEVEL3_PERIMETER AS (
        SELECT DISTINCT
            C.PRODUCT_CATEGORY_LEVEL3_ID,
            C.PRODUCT_CATEGORY_LEVEL4_ID
        FROM CATEGORY_LEVEL4_PERIMETER AS P
        INNER JOIN REFERENTIEL_SMT.SMT_RFL_DIM_PRODUCT_CATEGORY AS C
            ON P.PRODUCT_CATEGORY_LEVEL4_ID = C.PRODUCT_CATEGORY_LEVEL4_ID
            AND PRODUCT_CATEGORY_DEPTH = 4
    ),
    CATEGORY_LEVEL3_PATH_PERIMETER AS (
        SELECT
            P.PRODUCT_CATEGORY_LEVEL3_ID,
            P.PRODUCT_CATEGORY_LEVEL4_ID,
            C.PRODUCT_CATEGORY_CODE_PATH
        FROM CATEGORY_LEVEL3_PERIMETER AS P
        INNER JOIN REFERENTIEL_SMT.SMT_RFL_DIM_PRODUCT_CATEGORY AS C
            ON P.PRODUCT_CATEGORY_LEVEL3_ID = C.PRODUCT_CATEGORY_LEVEL3_ID
            AND PRODUCT_CATEGORY_DEPTH = 3
    ),
    TRAFFIC AS (
        SELECT
            SEARCH_ID,
            SUM(VIEW_COUNT::FLOAT) AS VIEW_COUNT,
            SUM(CLICK_COUNT) AS CLICK_COUNT,
            SUM(TURNOVER) AS TURNOVER
        FROM SEARCH_SMT.SMT_SCH_AGG_SEARCH_LIST_TRACKING_KPI
        WHERE SNAPSHOT_DATE BETWEEN CURRENT_DATE - 365 AND CURRENT_DATE - 1
            AND SITE_ID = 100
            AND AB_TESTING_GROUP = 'A'
            AND SEARCH_ID <> ''
            AND SEARCH_ID IS NOT NULL
        GROUP BY SEARCH_ID
    ),
    CUMULATIVE_TRAFFIC AS (
        SELECT
            SEARCH_ID,
            (
                SUM(VIEW_COUNT)
                OVER(
                    ORDER BY VIEW_COUNT DESC, TURNOVER DESC, CLICK_COUNT DESC
                    ROWS UNBOUNDED PRECEDING
                )
            ) / (
                SUM(VIEW_COUNT)
                OVER()
            ) AS CUM_QP_TRAFFIC
        FROM TRAFFIC
    ),
    TRAFFIC_BINS AS (
        SELECT
            SEARCH_ID,
            FLOOR(
                IFF(
                    CUM_QP_TRAFFIC = 1,
                    CUM_QP_TRAFFIC - 1e-5,
                    CUM_QP_TRAFFIC
                ) * 10
            ) AS SEARCH_ID_GROUP
        FROM CUMULATIVE_TRAFFIC
    ),
    VIEWED_PRODUCTS AS (
        SELECT
            SEARCH_ID,
            PRODUCT_ID,
            SUM(VIEW_COUNT) AS VIEW_COUNT
        FROM SEARCH_SMT.SMT_SCH_AGG_SEARCH_LIST_PRODUCT_VIEW
        WHERE SNAPSHOT_DATE BETWEEN CURRENT_DATE - 30 AND CURRENT_DATE - 1
            AND SITE_ID = 100
        GROUP BY
            SEARCH_ID,
            PRODUCT_ID
--        HAVING SUM(VIEW_COUNT) > 1
    ),
    CATEGORIZED_SEACH_ID AS (
        SELECT DISTINCT LR.SEARCH_ID AS SEARCH_ID
        FROM CATEGORY_LEVEL3_PATH_PERIMETER AS P
        INNER JOIN SEARCH_SMT.SMT_SCH_DIM_SEARCH_LIST_CATEGORY_FILTER AS LR
            ON P.PRODUCT_CATEGORY_CODE_PATH = LR.CATEGORY_FILTER_PATH_CODE
            AND LR.SNAPSHOT_DATE BETWEEN CURRENT_DATE - 365 AND CURRENT_DATE - 1
            AND LR.SITE_ID = 100
            AND LR.AB_TESTING_SVC_COOKIE_GROUP = 'A'
    ),
    CATEGORIZED_VIEWED_PRODUCTS AS (
        SELECT
            T.SEARCH_ID_GROUP,
            V.SEARCH_ID,
            V.PRODUCT_ID,
            SUM(V.VIEW_COUNT) AS VIEW_COUNT
            
        FROM CATEGORIZED_SEACH_ID AS LR
        INNER JOIN VIEWED_PRODUCTS AS V
            ON LR.SEARCH_ID = V.SEARCH_ID
        INNER JOIN TRAFFIC_BINS AS T
            ON LR.SEARCH_ID = T.SEARCH_ID
        GROUP BY
            T.SEARCH_ID_GROUP,
            V.SEARCH_ID,
            V.PRODUCT_ID
    )
SELECT
    SEARCH_ID_GROUP,
    SEARCH_ID,
    PRODUCT_ID,
    VIEW_COUNT
FROM CATEGORIZED_VIEWED_PRODUCTS
QUALIFY ROW_NUMBER() OVER (PARTITION BY SEARCH_ID_GROUP, SEARCH_ID ORDER BY VIEW_COUNT DESC) <= 100
;
    """, con=con)

In [4]:
lr_df

,search_id_group,search_id,product_id,view_count
0,3.0,canapeangle200cm,AUC3700538225186,235
1,3.0,canapeangle200cm,ASPENHDGTX,199
2,3.0,canapeangle200cm,A108301,192
3,3.0,canapeangle200cm,DES3760250736612,180
4,3.0,canapeangle200cm,BAI3584179069817,162
...,...,...,...,...
316739,7.0,canapeangledroitnoir,DEL4250809379272,2
316740,7.0,canapeangledroitnoir,VEN3517920877393,2
316741,7.0,canapeangledroitnoir,BES3701061722265,2
316742,7.0,canapeangledroitnoir,VEN4894223128414,2


In [4]:
full_df = julien_products_df.merge(lr_df, on="product_id")
full_df.rename(columns={'fp_product_name':'product_name', 'product_long_description':'description'}, inplace=True)
full_df.description.fillna("", inplace=True)
full_df.drop(columns=['product_short_description',
                      'product_properties',
                      'preprocessed_product_properties',
                      'product_marketing_description',
                      'total_token_fp_product_name',
                      'total_token_product_long_description',
                      'total_token_fp_content',
                      #'community_id'
                     ], inplace=True)
saving_csv = False
if saving_csv:
    full_df.to_csv("csv_files/lr_products.csv", index=False, sep='\u0001')

ANNOY-ING NEIGHBORS

In [5]:
def id2product_id(index):
    return full_df.iloc[index].product_id

def get_neighbors(u, i, k):
    neighbors, distances = u.get_nns_by_item(i=i, n=k+1, include_distances=True)
    return (
      pd.DataFrame({
          'id': i,
          'neighbor_id': neighbors,
          'distance': distances
      })
      .loc[lambda x: x.neighbor_id.ne(i)]
      .head(k)
      .assign(rank=[j for j in range(1, k+1)])
    )

In [6]:
index_list = full_df["index"]
vec_dim = 1536
original_annoy_index = AnnoyIndex(vec_dim, 'dot')
original_annoy_index.load('/home/jupyter/20231123_stage_vincent_products_fp_content.ann')
u = AnnoyIndex(vec_dim, 'dot')
new_idx = 0
for i in tqdm(index_list, total=len(index_list)):
    vector = original_annoy_index.get_item_vector(i)
    u.add_item(new_idx, vector)
    new_idx += 1
u.build(100)

100%|██████████| 275071/275071 [00:22<00:00, 12038.83it/s]


True

In [14]:
def get_products_similarity_score_median(ids, embedded_products: AnnoyIndex):
    similarities = []
    for id1 in ids:
        for id2 in ids:
            if id1 <= id2:
                break
            similarities.append(embedded_products.get_distance(id1, id2))
    return np.median(similarities)

In [17]:
product_id_to_id = {v: k for k, v in full_df.product_id.to_dict().items()}

In [19]:
similarity_scores = {}
for name, group in tqdm(full_df.groupby(['search_id_group', 'search_id'])):
    search_id_group, search_id = name
    selected_products = group.product_id
    ids = sorted([product_id_to_id[p] for p in selected_products])
    score = get_products_similarity_score_median(
        ids=ids,
        embedded_products=u
    )
    if search_id_group not in similarity_scores:
        similarity_scores[search_id_group] = {}
    similarity_scores[search_id_group][search_id] = score

 75%|███████▍  | 2976/3969 [00:05<00:01, 787.74it/s]/home/jupyter/.virtualenvs/icarusight/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jupyter/.virtualenvs/icarusight/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 3969/3969 [00:06<00:00, 614.61it/s]


In [44]:
{k: v for k, v in sorted(similarity_scores[1.0].items(), key=lambda item: item[1], reverse=True)}

{'canapeconvertibleexpress': 0.9398147761821747,
 'housseclicclac140x190': 0.9182742238044739,
 'canapechesterfield': 0.9174169898033142,
 'canapeangleconvertible': 0.9138427972793579,
 'houssecanape': 0.9133579134941101,
 'houssedecanape': 0.9124626219272614,
 'canapedanglebeige': 0.9107554852962494,
 'canapedangleconvertiblecoffre': 0.9106262028217316,
 'canaperelax': 0.9092907309532166,
 'canapepanoramique': 0.9060946404933929,
 'canapejosh': 0.90528404712677,
 'canapevelourscotele': 0.9047065079212189,
 'canapemira': 0.901705265045166,
 'canapedroit': 0.9004745483398438,
 'canape2placesconvertible': 0.8986257612705231,
 'canapebeige': 0.8962932825088501,
 'canape2place': 0.8958171606063843,
 'canapevelours': 0.8957047760486603,
 'sofa': 0.8947322368621826,
 'canapepoltronesofa': 0.8930362462997437,
 'canapes': 0.8923382759094238,
 'canape3place': 0.8909438848495483,
 'canapecuir': 0.8903276920318604}

In [35]:
def look4one_search_id(search_id):
    return (full_df
        .loc[lambda x: x.search_id.eq(search_id) & x.product_id.isin(full_df.product_id), 'product_name']
        .tolist())

In [45]:
look4one_search_id('canapeconvertibleexpress')

['Canapé convertible EXPRESS 3/4 places en polyuréthane noir - Couchage 160cm - Epaisseur matelas 16cm - MIDNIGHT',
 'Canapé convertible EXPRESS 2/3 places en tissu gris graphite - Couchage 120cm - Epaisseur matelas 16cm - MIDNIGHT',
 'Canapé convertible EXPRESS 2/3 places en velours bleu pastel - Couchage 120cm - Matelas épaisseur 22cm à mémoire de forme - COLOSSE',
 'Canapé convertible express SUN LIMITED 140 cm matelas 14 cm velours vert vert Velours Inside75',
 'Canapé convertible EXPRESS 2/3 places en cuir et polyuréthane blanc - Couchage 120cm - Epaisseur matelas 16cm - DREAMER EDITION CUIR',
 'Canapé convertible EXPRESS 2/3 places en tissu tweed taupe - Couchage 120cm - Epaisseur matelas 18cm - ECLIPSE',
 'Canapé convertible EXPRESS 3 places en cuir et polyuréthane blanc - Couchage 140cm - Epaisseur matelas 16cm - DREAMER EDITION CUIR',
 'Canapé convertible EXPRESS 3 places en velours bleu pastel - Couchage 140cm - Matelas épaisseur 22cm à mémoire de forme - COLOSSE',
 "Canapé d

In [9]:
full_df.columns

Index(['index', 'product_id', 'brand_name', 'product_name', 'description',
       'fp_content', 'search_id_group', 'search_id', 'view_count'],
      dtype='object')

In [ ]:
def check_products_in_lr(search_id):
    rows = exploded_lr_df.loc[exploded_lr_df.search_id == search_id]
    for product_id in rows.product_id:
        if product_id in full_df.product_id.tolist():
            display(full_df[full_df.product_id == product_id].product_name.item())

In [ ]:
check_products_in_lr('canapeelectrique')